In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from plotly.io import show
from datetime import datetime
from sklearn.model_selection import train_test_split

from skfolio import Population, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import (
    InverseVolatility,
    MeanRisk,
    ObjectiveFunction,
    EqualWeighted,
)
from skfolio.preprocessing import prices_to_returns

In [13]:
prices = load_sp500_dataset()
X = prices_to_returns(prices)
X.head()

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
1990-01-03,0.007576,-0.030303,0.008045,0.118056,-0.016229,-0.001876,0.003581,0.004072,0.033589,-0.014318,0.000000,0.015896,0.005208,-0.009709,0.002938,-0.001813,0.0,-0.019355,0.000000,-0.010079
1990-01-04,0.003759,-0.015500,-0.021355,-0.012422,-0.012831,-0.005639,0.006244,0.002028,0.003991,-0.004993,-0.005557,-0.015647,0.028497,-0.009804,0.016602,-0.019725,0.0,-0.009868,-0.005201,-0.009933
1990-01-05,0.003745,-0.031996,-0.021821,0.000000,-0.014855,-0.009452,-0.013298,-0.010408,0.003975,-0.008212,-0.010874,-0.020641,-0.025189,-0.013991,-0.008646,-0.018004,0.0,-0.043189,-0.010732,-0.005267
1990-01-08,0.003731,0.000000,0.005633,-0.075472,0.009424,0.005725,-0.009883,0.016944,0.000000,0.021159,0.000000,0.012839,0.015504,0.018118,-0.008721,0.018334,0.0,-0.020833,0.013630,0.015381
1990-01-09,-0.007435,0.016527,0.000000,0.000000,-0.007469,-0.020803,-0.026316,-0.031026,-0.031957,-0.007658,-0.011147,-0.007893,-0.002545,-0.013722,-0.021505,0.000000,0.0,-0.024823,-0.026619,-0.020114


In [16]:
X_month = (1 + X).resample('ME').prod() - 1
X_month.head()

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
1990-01-31,-0.087121,-0.121212,-0.099369,0.104167,-0.032458,-0.067403,-0.069830,-0.125364,-1.414260e-01,-0.117226,-0.081856,-0.081139,0.041667,-0.105108,-0.035260,-0.086528,0.000000,-0.245161,-0.095538,-0.059980
1990-02-28,0.004149,0.137931,0.068083,0.125786,0.014289,-0.000447,0.132820,0.029265,-5.147563e-03,0.023822,-0.037461,-0.053705,0.067500,0.017453,-0.130964,-0.001985,0.000000,0.012821,0.032385,0.012552
1990-03-31,0.181818,0.105939,-0.129069,0.335196,0.005512,0.040477,0.151232,0.046527,-6.450500e-02,0.066337,0.070530,0.023192,0.121780,0.075104,0.026869,0.060813,0.000000,0.042194,0.075344,-0.015754
1990-04-30,-0.020979,-0.040991,-0.019465,-0.217573,-0.023959,-0.003940,0.021402,0.008953,4.440892e-16,0.019963,0.023178,0.053867,0.045929,0.057999,-0.063709,0.049022,-0.166767,0.101215,0.050436,-0.021779
1990-05-31,0.050000,0.200000,0.029116,0.310160,0.073851,0.083507,0.296965,0.134333,1.710914e-01,0.183432,0.190380,0.135881,0.259481,0.115142,0.172539,0.169816,0.600072,0.408088,0.135998,0.074571


In [17]:
X_train, X_test = train_test_split(X_month, test_size=0.3, shuffle=False)
X_train.head()

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
1990-01-31,-0.087121,-0.121212,-0.099369,0.104167,-0.032458,-0.067403,-0.069830,-0.125364,-1.414260e-01,-0.117226,-0.081856,-0.081139,0.041667,-0.105108,-0.035260,-0.086528,0.000000,-0.245161,-0.095538,-0.059980
1990-02-28,0.004149,0.137931,0.068083,0.125786,0.014289,-0.000447,0.132820,0.029265,-5.147563e-03,0.023822,-0.037461,-0.053705,0.067500,0.017453,-0.130964,-0.001985,0.000000,0.012821,0.032385,0.012552
1990-03-31,0.181818,0.105939,-0.129069,0.335196,0.005512,0.040477,0.151232,0.046527,-6.450500e-02,0.066337,0.070530,0.023192,0.121780,0.075104,0.026869,0.060813,0.000000,0.042194,0.075344,-0.015754
1990-04-30,-0.020979,-0.040991,-0.019465,-0.217573,-0.023959,-0.003940,0.021402,0.008953,4.440892e-16,0.019963,0.023178,0.053867,0.045929,0.057999,-0.063709,0.049022,-0.166767,0.101215,0.050436,-0.021779
1990-05-31,0.050000,0.200000,0.029116,0.310160,0.073851,0.083507,0.296965,0.134333,1.710914e-01,0.183432,0.190380,0.135881,0.259481,0.115142,0.172539,0.169816,0.600072,0.408088,0.135998,0.074571


In [18]:
# use skfolio to implement MVO strategy
# use max sharpe ratio as the objective function
# for mean just use simlpe mean estimator, with alternative as stein shrinkage estimator
# for cov mat use ledoit wolf shrinkage estimator

from skfolio.prior import EmpiricalPrior
from skfolio.moments import EmpiricalMu, ShrunkMu, LedoitWolf, DenoiseCovariance

model = MeanRisk(
    risk_measure=RiskMeasure.STANDARD_DEVIATION,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    prior_estimator=EmpiricalPrior(
        mu_estimator=EmpiricalMu(window_size=None),  # by default uses all given data
        covariance_estimator=LedoitWolf(),
    ),
    # note that default uses empirical prior -> ie empirical mu and cov
    portfolio_params=dict(name="Max Sharpe"),
)
model.fit(X_train)
model.weights_

array([6.81716932e-02, 1.24919732e-08, 1.89022862e-08, 7.43892024e-02,
       4.89858527e-02, 3.92019171e-08, 7.39197772e-02, 4.45494127e-02,
       4.09377087e-08, 2.63762871e-02, 3.90372986e-03, 5.14911303e-07,
       3.42727749e-02, 1.87166355e-06, 3.83480882e-02, 1.75528317e-01,
       4.96540315e-02, 1.10553000e-01, 6.48071143e-02, 1.86538221e-01])

In [20]:
# model.prior_estimator_.mu_estimator_.window_size
# model.prior_estimator_.covariance_estimator_.shrinkage_

# model.prior_estimator_.return_distribution_
# model.prior_estimator_.return_distribution_.covariance

In [21]:
# compare against inverse vol and baseline 1/N allocation
# with rebalancing each period to make sure that
# the uniform allocation is up to date to the current prices

benchmark = InverseVolatility(portfolio_params=dict(name="Inverse Vol"))
benchmark.fit(X_train)
benchmark.weights_

array([0.02926877, 0.02045975, 0.03430406, 0.02264698, 0.07035864,
       0.05383435, 0.04886095, 0.06845916, 0.03954924, 0.06450685,
       0.05177101, 0.05253098, 0.04155939, 0.06681327, 0.05706484,
       0.06722756, 0.02566855, 0.04082515, 0.05970715, 0.08458335])

In [22]:
naive = EqualWeighted(portfolio_params=dict(name="Naive"))
naive.fit(X_train)
naive.weights_

array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
       0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

In [23]:
pred_model = model.predict(X_test)
pred_bench = benchmark.predict(X_test)
pred_naive = naive.predict(X_test)

print(pred_model.annualized_sharpe_ratio)
print(pred_bench.annualized_sharpe_ratio)
print(pred_naive.annualized_sharpe_ratio)

4.693442512496158
4.91937810039147
5.06644192550086


In [24]:
population = Population([pred_model, pred_bench, pred_naive])
population.plot_composition()

In [25]:
fig = population.plot_cumulative_returns()
# show(fig) is only used for the documentation sticker.
show(fig)

In [12]:
population.summary()

,Max Sharpe,Inverse Vol
Mean,0.076%,0.064%
Annualized Mean,19.03%,16.05%
Variance,0.012%,0.011%
Annualized Variance,3.14%,2.68%
Semi-Variance,0.0063%,0.0055%
Annualized Semi-Variance,1.58%,1.40%
Standard Deviation,1.12%,1.03%
Annualized Standard Deviation,17.73%,16.36%
Semi-Deviation,0.79%,0.74%
Annualized Semi-Deviation,12.57%,11.82%
